In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_210714/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 11:53:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/14 11:53:54 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [11]:
import pandas as pd

# Sample data
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4],
    'spend_date': ['2019-07-01', '2019-07-01', '2019-07-01', '2019-07-02', '2019-07-01', '2019-07-02', '2019-07-01', '2019-07-01'],
    'platform': ['mobile', 'desktop', 'mobile', 'mobile', 'desktop', 'desktop', 'mobile', 'desktop'],
    'amount': [100, 100, 100, 100, 100, 100, 200, 300]
}

# Create DataFrame
df = pd.DataFrame(data)

print(df)

df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Spending")



   user_id  spend_date platform  amount
0        1  2019-07-01   mobile     100
1        1  2019-07-01  desktop     100
2        2  2019-07-01   mobile     100
3        2  2019-07-02   mobile     100
4        3  2019-07-01  desktop     100
5        3  2019-07-02  desktop     100
6        4  2019-07-01   mobile     200
7        4  2019-07-01  desktop     300


In [ ]:
query_0 = """
    
"""

In [30]:
query = """
    with t as (
        select
            spend_date,
            platform,
            sum(total_amount) as total_amount,
            count(distinct user_id) as total_users
        from (
            select
                distinct
                user_id,
                spend_date,
                platform_count as platform,
                total_amount
            from (
                select
                    *,
                    case 
                        when count(platform) over (partition by spend_date, user_id) = 2 then 'both' 
                        else platform
                    end as platform_count,
                    sum(amount) over (partition by spend_date, user_id) as total_amount
                from Spending s 
            ) 
        ) group by spend_date, platform
    )
    
    select
        k.spend_date,
        e.platform,
        ifnull(t.total_amount, 0),
        ifnull(t.total_users, 0)
    from (
        select distinct spend_date from t
    ) k cross join (
        select "both" as platform
        union
        select "mobile" as platform
        union
        select "desktop" as platform
    ) e left join t on k.spend_date = t.spend_date and e.platform = t.platform
    order by k.spend_date, e.platform
"""

In [31]:
spark.sql(query).show()

+----------+--------+-----------------------+----------------------+
|spend_date|platform|ifnull(total_amount, 0)|ifnull(total_users, 0)|
+----------+--------+-----------------------+----------------------+
|2019-07-01|    both|                    700|                     2|
|2019-07-01| desktop|                    100|                     1|
|2019-07-01|  mobile|                    100|                     1|
|2019-07-02|    both|                      0|                     0|
|2019-07-02| desktop|                    100|                     1|
|2019-07-02|  mobile|                    100|                     1|
+----------+--------+-----------------------+----------------------+



In [22]:
test_query = """
    select "both" as platform
    union
    select "mobile" as platform
    union
    select "desktop" as platform
"""

In [23]:
spark.sql(test_query).show()

+--------+
|platform|
+--------+
|    both|
|       m|
|       d|
+--------+



In [18]:
query_1 = """

        select
            distinct
            user_id,
            spend_date,
            platform_count as platform,
            total_amount
        from (
            select
                *,
                case 
                    when count(platform) over (partition by spend_date, user_id) = 2 then 'both' 
                    else platform
                end as platform_count,
                sum(amount) over (partition by spend_date, user_id) as total_amount
            from Spending s 
        ) 
"""

In [19]:
spark.sql(query_1).show()

+-------+----------+--------+------------+
|user_id|spend_date|platform|total_amount|
+-------+----------+--------+------------+
|      1|2019-07-01|    both|         200|
|      2|2019-07-01|  mobile|         100|
|      3|2019-07-01| desktop|         100|
|      4|2019-07-01|    both|         500|
|      2|2019-07-02|  mobile|         100|
|      3|2019-07-02| desktop|         100|
+-------+----------+--------+------------+

